### Import

In [1]:
import pandas as pd
import numpy as np
import gc

import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

### Data Load

### Data Preprocessing(1) : Concat & Merge

In [2]:
# 데이터 유형별 설정 
info_categories = ["customer", "credit", "sales", "billing", "balance", "channel", "marketing", "performance"]

# 월 설정
months = ['07', '08', '09', '10', '11', '12']

### Data Preprocessing(2) : Encoding

In [3]:
train_df = pd.read_parquet("./data/train/merge_train.parquet")
test_df = pd.read_parquet("./data/test/merge_test.parquet")

In [4]:
date_feature = train_df.select_dtypes(include=["datetime64"]).columns.tolist()
feature_cols = train_df.columns.tolist()
feature_cols = [feature for feature in feature_cols if feature not in date_feature + ["Segment"]]

X = train_df[feature_cols].copy()
y = train_df["Segment"].copy()

# 타깃 라벨 인코딩
le_target = LabelEncoder()
y_encoded = le_target.fit_transform(y)

In [5]:
categorical_features = X.select_dtypes(include=['object']).columns.tolist()

X_test = test_df.copy()

encoders = {}  # 각 컬럼별 encoder 저장

for col in categorical_features:
    le_train = LabelEncoder()
    X[col] = le_train.fit_transform(X[col])
    encoders[col] = le_train
    unseen_labels_val = set(X_test[col]) - set(le_train.classes_)
    if unseen_labels_val:
        le_train.classes_ = np.append(le_train.classes_, list(unseen_labels_val))
    X_test[col] = le_train.transform(X_test[col])

In [6]:
gc.collect()

0

### Train

In [ ]:
try:
    model = xgb.XGBClassifier(
        tree_method='gpu_hist',  # GPU 모드 설정
        device=0,
        random_state=42
    )
    print("GPU 사용 가능: gpu_hist 모드 적용")
    model.fit(X, y_encoded)
    
except Exception:
    model = xgb.XGBClassifier(
        random_state=42
    )
    print("GPU 사용 불가: CPU 모드 적용")
    model.fit(X, y_encoded)

GPU 사용 가능: gpu_hist 모드 적용


/Users/argus/.pyenv/versions/DACON_CreditSegmentClassification/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [19:10:13] WARNING: /Users/runner/work/xgboost/xgboost/src/common/error_msg.cc:45: `gpu_id` is deprecated since2.0.0, use `device` instead. E.g. device=cpu/cuda/cuda:0
  bst.update(dtrain, iteration=i, fobj=obj)
/Users/argus/.pyenv/versions/DACON_CreditSegmentClassification/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [19:10:13] WARNING: /Users/runner/work/xgboost/xgboost/src/context.cc:203: XGBoost is not compiled with CUDA support.
  bst.update(dtrain, iteration=i, fobj=obj)


GPU 사용 불가: CPU 모드 적용


: 

In [1]:
model.fit(X, y_encoded)

NameError: name 'model' is not defined

### Predict

In [ ]:
X_test.drop(columns=['ID'],inplace=True)

In [ ]:
# row-level 예측 수행
y_test_pred = model.predict(X_test)
# 예측 결과를 변환
y_test_pred_labels = le_target.inverse_transform(y_test_pred)

# row 단위 예측 결과를 test_data에 추가
test_data = test_df.copy()  # 원본 유지
test_data["pred_label"] = y_test_pred_labels

### Submission

In [ ]:
submission = test_data.groupby("ID")["pred_label"] \
    .agg(lambda x: x.value_counts().idxmax()) \
    .reset_index()

submission.columns = ["ID", "Segment"]

In [ ]:
submission.to_csv('./base_submit.csv',index=False)